In [1]:
import gdown

In [2]:
file_url="https://drive.google.com/drive/folders/1Kcnsas1hx3t361WxjiULyOdWg1DkoLE9?usp=drive_link"

In [7]:
file_url.split("/")[-1].split("?")[0]

'1Kcnsas1hx3t361WxjiULyOdWg1DkoLE9'

In [3]:
%pwd

'e:\\MLOPS-E2E-Kidney_Classification\\research'

In [1]:
import os,sys

In [2]:
%pwd

'e:\\MLOPS-E2E-Kidney_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
from src.CNN import components,config,constants

In [5]:
from src.CNN.utils.common import read_yaml, create_directories, save_json
from src.CNN.entity.config_entity import PrepareBaseModelConfig
# from src.CNN.config.configuration import ConfigManager
from src.CNN.constants import *

In [6]:
import numpy as np

In [7]:
from src.CNN.entity.config_entity import TrainConfigs

In [8]:
class ModelConfig:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def prepare_data_for_model(self)->TrainConfigs:
        config=self.config.model_train
        
        return TrainConfigs(
            config.model_file_path,
            config.data_file_path,
            config.train_history,
            config.save_weights_path,
            self.params.IMAGE_SIZE,
            self.params.BATCH_SIZE,
            self.params.EPOCHS,
            self.params.CLASSES,
            self.params.featurewise_center,
            self.params.featurewise_std_normalization,
            self.params.rotation_range,
            self.params.width_shift_range,
            self.params.height_shift_range,
            self.params.horizontal_flip,
            self.params.validation_split,
            self.params.rescale,
        )

In [9]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.CNN import logger

In [10]:
from glob import glob
import cv2
import json

In [11]:
from tensorflow.keras.models import load_model
from keras.utils import to_categorical

In [20]:
import mlflow
import mlflow.keras

In [13]:
data=[]
class_name=[]

In [29]:
class TrainModel:
    def __init__(self, config: TrainConfigs):
        self.config = config
        self.data=[]
        self.class_name=[]
        
    def train(self):
        mlflow.keras.autolog()
        with mlflow.start_run():
            self.load_data(self,self.config.data_file_path)
            train,valid=self.preprocess_data(self)
            model=load_model('./artifacts/prepare_base_model/base_model_update.h5')
            logger.info("Model is loaded from {}".format(self.config.model_file_path))
            self.hist=model.fit(
                train,
                epochs=self.config.EPOCHS,
                validation_data=(valid),
            )
            # mlflow.log_params(params=dict(self.config.__dict__))
            model.save('./'+self.config.save_weights_path+'/weights.h5')
            mlflow.keras.log_model(model, "MobileNetV2_Train")
            with open('./'+self.config.train_history, "w") as f:
                json.dump(self.hist.history, f)
                logger.info("Saving history at {}".format(self.config.train_history))
            # mlflow.log_metrics(dict(self.hist.history))
            logger.info("Model is saved at {}".format(self.config.model_file_path))
        # print("Model is saved at {}".format(self.config.model_file_path))
    @staticmethod
    def preprocess_data(self,augmentation=True):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
            self.data, self.class_name, test_size=0.2, random_state=42
        )
        self.y_train = to_categorical(self.y_train, num_classes=self.config.CLASSES)
        self.y_test = to_categorical(self.y_test, num_classes=self.config.CLASSES)
        # print(self.x_train[0].shape)
        datagen = ImageDataGenerator(
            featurewise_center=self.config.featurewise_center,
            featurewise_std_normalization=self.config.featurewise_std_normalization,
            rotation_range=self.config.rotation_range,
            width_shift_range=self.config.width_shift_range,
            height_shift_range=self.config.height_shift_range,
            horizontal_flip=self.config.horizontal_flip,
            rescale=self.config.rescale,
        )
        validgen = ImageDataGenerator(
            featurewise_center=self.config.featurewise_center,
            featurewise_std_normalization=self.config.featurewise_std_normalization,
            rotation_range=self.config.rotation_range,
            width_shift_range=self.config.width_shift_range,
            height_shift_range=self.config.height_shift_range,
            horizontal_flip=self.config.horizontal_flip,
            rescale=self.config.rescale,
        )
        try:
            datagen.fit(self.x_train)
            validgen.fit(self.x_test)
            # print(dir(train_gen.flow))
            print("Data Augmentation Done")
            logger.info("Data Augmentation Done")
            self.x_train=np.array(self.x_train)
            self.x_test=np.array(self.x_test)
            self.y_train=np.array(self.y_train)
            self.y_test=np.array(self.y_test)
            train_data=datagen.flow(self.x_train,self.y_train)
            valid_data=validgen.flow(self.x_test,self.y_test)
            return train_data,valid_data    
        except Exception as e:
            print(e)
            # logger.info(e)
        # print(x_test_processed[0])
        
    @staticmethod
    def load_data(self,data):
        for index,folder in enumerate(glob("./"+self.config.data_file_path+"/*")):
            for image in glob(folder+"/*.jpg"):
                img = cv2.imread(image)
                img = cv2.resize(img, tuple(self.config.IMAGE_SIZE[:-1]), interpolation=cv2.INTER_AREA)
                img = np.array(img)
                self.data.append(img)
                self.class_name.append(index)
        print("Data Loaded")
        logger.info("Data Loaded")

In [30]:
try:
    model_config = ModelConfig()
    train_config = model_config.prepare_data_for_model()
    train_model = TrainModel(train_config)
    # print(dict(train_model.config.__dict__))
    mlflow.set_tracking_uri(uri="https://dagshub.com/SHREYAS290601/E2E-MLFLOW-DVC-Kidney_classification.mlflow")
    mlflow.set_experiment("Default")
    train_model.train()
except Exception as e:
    logger.exception(e)
    raise e

[2024-01-02 14:59:43,971 : INFO : common : Yaml File : config\config.yaml Loaded Successfully : ]
[2024-01-02 14:59:43,979 : INFO : common : Yaml File : params.yaml Loaded Successfully : ]
[2024-01-02 14:59:43,982 : INFO : common : Created Directory : artifacts : ]
Data Loaded
[2024-01-02 14:59:53,441 : INFO : 1200640191 : Data Loaded : ]
Data Augmentation Done
[2024-01-02 14:59:59,457 : INFO : 1200640191 : Data Augmentation Done : ]
[2024-01-02 15:00:01,985 : INFO : 1200640191 : Model is loaded from artifacts/prepare_base_model/base_model_update.h5 : ]


2024/01/02 15:00:03 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.NumpyArrayIterator'>. Dataset logging skipped.
2024/01/02 15:00:03 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'keras.preprocessing.image.NumpyArrayIterator'>. Dataset logging skipped.


25/25 [==============================] - 34s 1s/step - loss: 55.4224 - accuracy: 0.6148 - val_loss: 17.6651 - val_accuracy: 0.7876


2024/01/02 15:00:44 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'
2024/01/02 15:00:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-02 15:00:58,408 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading. : ]
[2024-01-02 15:01:02,128 : INFO : builder_impl : Assets written to: C:\Users\Shreyas\AppData\Local\Temp\tmpj6bcd5es\model\data\model\assets : ]


2024/01/02 15:01:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-02 15:02:05,473 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading. : ]
[2024-01-02 15:02:09,078 : INFO : builder_impl : Assets written to: C:\Users\Shreyas\AppData\Local\Temp\tmp0ach15wa\model\data\model\assets : ]
[2024-01-02 15:02:38,731 : INFO : 1200640191 : Saving history at artifacts/model_train/train_history.json : ]
[2024-01-02 15:02:38,735 : INFO : 1200640191 : Model is saved at artifacts/prepare_base_model/base_model_update.h5 : ]


import os,sys
%pwd
os.chdir("../")
from src.CNN import components,config,constants
from src.CNN.utils.common import read_yaml, create_directories, save_json
from src.CNN.entity.config_entity import PrepareBaseModelConfig
# from src.CNN.config.configuration import ConfigManager
from src.CNN.constants import *
import numpy as np
from src.CNN.entity.config_entity import TrainConfigs
class ModelConfig:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def prepare_data_for_model(self)->TrainConfigs:
        config=self.config.model_train
        
        return TrainConfigs(
            config.model_file_path,
            config.data_file_path,
            config.train_history,
            config.save_weights_path,
            self.params.IMAGE_SIZE,
            self.params.BA

In [1]:
import os,sys

In [2]:
%pwd

'e:\\MLOPS-E2E-Kidney_Classification\\research'

In [3]:
os.chdir("../")

In [18]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/SHREYAS290601/E2E-MLFLOW-DVC-Kidney_classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="SHREYAS290601"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b06e74ec71a76270575d2e406cbc1d9f87f6a357"

In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import mlflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
model=load_model('./artifacts/model_train/model_weights/weights.h5')

In [7]:
from src.CNN.entity.config_entity import *
from src.CNN.utils.common import *

In [14]:
class EvalModelManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def evaluate_model_config(self) -> EvalConfig:
        config = self.config.model_eval
        # print(config)
        # create_directories([config.root_dir])

        return EvalConfig(
            path_to_model=config.model_weight_path,
            training_data=config.data_file_path,
            mlflow_uri=os.getenv("MLFLOW_TRACKING_URI"),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch=self.params.BATCH_SIZE,
        )

In [10]:
from sklearn.utils import shuffle

In [47]:
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from glob import glob
import cv2
from keras.utils import to_categorical

In [50]:
class EvaluateModel:
    def __init__(self, config:EvalConfig):
        self.config=config
        self.data=[]
        self.class_name=[]
    
    def evaluate(self):
        self.get_eval_data(self)
        evaldata=self.create_eval_data(self)
        self.model = load_model("./artifacts/model_train/model_weights/weights.h5")
        logger.info(f"Model is loaded from {self.config.path_to_model}")
        self.score=self.model.evaluate(evaldata)
        print("Loss: ",self.score[0])
        print("Accuracy: ",self.score[1])
        logger.info(f"Loss: {self.score[0]}")
        logger.info(f"Accuracy: {self.score[1]}")
        logger.info(f"Accuracy is {self.score[1]*100}%")
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="MobileNetV2")
            else:
                mlflow.keras.log_model(self.model, "model")
    
    @staticmethod
    def get_eval_data(self):
        for index,folder in enumerate(glob(f"./{self.config.training_data}/*")):
            for _,image in enumerate(glob(f"{folder}/*.jpg")):
                if _<=20:
                    img = cv2.imread(image)
                    img = cv2.resize(img, tuple(self.config.params_image_size[:-1]), interpolation=cv2.INTER_AREA)
                    img = np.array(img)
                    self.data.append(img)
                    self.class_name.append(index)
        print("Data Loaded")
        logger.info("Data Loaded")
    
    
    @staticmethod
    def create_eval_data(self):
        evalgen=ImageDataGenerator(
            featurewise_center=self.config.all_params.featurewise_center,
            featurewise_std_normalization=self.config.all_params.featurewise_std_normalization,
            rotation_range=self.config.all_params.rotation_range,
            width_shift_range=self.config.all_params.width_shift_range,
            height_shift_range=self.config.all_params.height_shift_range,
            horizontal_flip=self.config.all_params.horizontal_flip,
            rescale=self.config.all_params.rescale,
            
        )
        evalgen.fit(self.data)
        self.data=np.array(self.data)
        self.class_name=np.array(to_categorical(self.class_name,num_classes=self.config.all_params.CLASSES))
        return evalgen.flow(self.data,self.class_name)
            

In [51]:
try:
    config = EvalModelManager()
    eval_config = config.evaluate_model_config()
    evaluation = EvaluateModel(eval_config)
    # print(eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-01-01 19:44:17,062 : INFO : common : Yaml File : config\config.yaml Loaded Successfully : ]
[2024-01-01 19:44:17,065 : INFO : common : Yaml File : params.yaml Loaded Successfully : ]
[2024-01-01 19:44:17,066 : INFO : common : Created Directory : artifacts : ]
Data Loaded
[2024-01-01 19:44:17,742 : INFO : 1063786012 : Data Loaded : ]
[2024-01-01 19:44:19,655 : INFO : 1063786012 : Model is loaded from artifacts/model_train/model_weights/weigths.h5 : ]
3/3 [==============================] - 3s 700ms/step - loss: 1.6397 - accuracy: 0.9643
Loss:  1.6396616697311401
Accuracy:  0.9642857313156128
[2024-01-01 19:44:23,523 : INFO : 1063786012 : Loss: 1.6396616697311401 : ]
[2024-01-01 19:44:23,529 : INFO : 1063786012 : Accuracy: 0.9642857313156128 : ]
[2024-01-01 19:44:23,536 : INFO : 1063786012 : Accuracy is 96.42857313156128% : ]


2024/01/01 19:44:24 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-01 19:44:37,742 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 53). These functions will not be directly callable after loading. : ]
INFO:tensorflow:Assets written to: C:\Users\Shreyas\AppData\Local\Temp\tmpx7oce_iq\model\data\model\assets
[2024-01-01 19:44:42,129 : INFO : builder_impl : Assets written to: C:\Users\Shreyas\AppData\Local\Temp\tmpx7oce_iq\model\data\model\assets : ]


Successfully registered model 'MobileNetV2'.
2024/01/01 19:45:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: MobileNetV2, version 1
Created version '1' of model 'MobileNetV2'.
